In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('example.db')
def sql(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)

In [30]:
df = pd.read_csv('./summer_games.csv')
df.to_sql('summer_games', conn, if_exists='replace', index=False)

4937

In [20]:
sql("""-- Query the sport and distinct number of athletes
SELECT sport, COUNT(DISTINCT athlete_id) as athletes 
FROM summer_games 
GROUP BY sport
ORDER BY athletes DESC 
LIMIT 3;""")

,sport,athletes
0,Track and Field,2269
1,Swimming,942
2,Gymnastics,196


In [21]:
sql("""-- Query sport, events, and athletes from summer_games
SELECT 
	sport, 
    COUNT(DISTINCT event) AS events, 
    COUNT(DISTINCT athlete_id) AS athletes
FROM summer_games
GROUP BY sport;""")

,sport,events,athletes
0,Gymnastics,14,196
1,Swimming,34,942
2,Track and Field,47,2269


In [22]:
sql("""-- Select the age of the oldest athlete for each region
SELECT 
	region, 
    MAX(age) AS age_of_oldest_athlete
FROM athletes AS a
-- First JOIN statement
JOIN summer_games AS s
ON a.id = s.athlete_id
-- Second JOIN statement
JOIN countries AS c
ON s.country_id = c.id
GROUP BY region;""")

,region,age_of_oldest_athlete
0,None,36
1,ASIA (EX. NEAR EAST),44
2,BALTICS,39
3,C.W. OF IND. STATES,41
4,EASTERN EUROPE,39
5,LATIN AMER. & CARIB,42
6,NEAR EAST,40
7,NORTHERN AFRICA,41
8,NORTHERN AMERICA,41
9,OCEANIA,40


In [23]:
sql("""-- Select sport and events for summer sports
SELECT 
	sport, 
    COUNT(DISTINCT event) AS events
FROM summer_games
GROUP BY sport

UNION

-- Select sport and events for winter sports
SELECT 
	sport, 
    COUNT(DISTINCT event) AS events
FROM winter_games
GROUP BY sport

-- Show the most events at the top of the report
ORDER BY events DESC;""")

,sport,events
0,Track and Field,47
1,Swimming,34
2,Gymnastics,14
3,Cross Country Skiing,12
4,Biathlon,11
5,Alpine Skiing,10


In [39]:
sql("""SELECT *,
    CASE WHEN medal='Gold' THEN 1 ELSE NULL END AS gold,
    CASE WHEN medal='Silver' THEN 1 ELSE NULL END AS silver,
    CASE WHEN medal='Bronze' THEN 1 ELSE NULL END AS bronze 
    FROM winter_games""").to_sql('winter_games', conn, if_exists='replace', index=False)


2178

In [32]:
sql("""-- Add the rows column to your query
SELECT 
	bronze, 
	count(*) AS rows
FROM summer_games
GROUP BY bronze;""")

,bronze,rows
0,NaN,4796
1,1.0,141


In [34]:
sql(""" --Pull total_bronze_medals from summer_games below
SELECT SUM(bronze) AS total_bronze_medals
FROM summer_games; 
-->> OUTPUT = 141 total_bronze_medals */"""
    )

,total_bronze_medals
0,141.0


In [35]:
sql("""/* Pull total_bronze_medals from summer_games below
SELECT SUM(bronze) AS total_bronze_medals
FROM summer_games; 
>> OUTPUT = 141 total_bronze_medals */

-- Select the total bronze_medals from your query
SELECT SUM(bronze_medals)
FROM 
-- Previous query is shown below.  Alias this AS subquery.
  (SELECT 
      country, 
      SUM(bronze) AS bronze_medals
  FROM summer_games AS s
  JOIN countries AS c
  ON s.country_id = c.id
  GROUP BY country) AS subquery
;""")

,SUM(bronze_medals)
0,141.0
